# Loan Approval Prediction: A Machine Learning Pipeline

This notebook demonstrates a complete end-to-end machine learning pipeline for predicting loan approval status. We traverse through data acquisition, pre-processing, feature engineering, and model evaluation using **scikit-learn** and **pandas**.

## 1. Data Acquisition & Initial Setup

In this phase, we load binary libraries and the raw data. We also perform initial cleaning to ensure the data is in a usable format.

### Importing Core Libraries

**What is happening:** Loading the fundamental data science stack.

**The "Why":** We utilize **pandas** and **numpy** for data manipulation, while **matplotlib** and **seaborn** handle visualizations. **scikit-learn** provides the necessary tools for pre-processing, splitting, and modeling.

**Best Practices:** Grouping all imports at the top of the notebook ensures clear dependency management and improves readability.

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

### Loading and Cleaning the Dataset

**What is happening:** Reading the CSV file and removing unnecessary columns.

**The "Why":** Removing trailing spaces from column names ensures consistent indexing. Dropping `loan_id` is essential as unique identifiers do not contribute to the model's predictive power.

**Best Practices:** Identifying and removing unique ID columns prevents the model from 'memorizing' specific rows, which helps in better generalization.

In [111]:
df = pd.read_csv('loan_approval_dataset.csv')

# Clean column names by stripping whitespace
df.columns = df.columns.str.strip()

# Remove unique identifiers
df = df.drop(columns=['loan_id'])

## 2. Exploratory Data Analysis (EDA)

EDA is a critical step to understand the distribution of the target variable and the statistical properties of the features.

### Analyzing Target Class Proportions

**What is happening:** Calculating the percentage of approved vs. rejected loans.

**The "Why":** Understanding class balance is crucial. If one class significantly outweighs the other, accuracy can be a misleading metric.

**Best Practices:** Check class proportions early to determine if sampling techniques (like SMOTE) or adjusted evaluation metrics are needed.

In [112]:
count = np.array(df.value_counts(['loan_status']))
print('proportion:')
print(f'class 1: {100 * count[0] / np.sum(count):.2f}')
print(f'class 2: {100 * count[1] / np.sum(count):.2f}')

proportion:
class 1: 62.22
class 2: 37.78


### Inspecting Feature Names

**What is happening:** Printing the current column index.

**The "Why":** Verifying feature names ensures we are tracking all variables correctly after the initial cleanup.

In [113]:
print(df.columns, len(df.columns))

Index(['no_of_dependents', 'education', 'self_employed', 'income_annum',
       'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value',
       'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value',
       'loan_status'],
      dtype='object') 12


### Statistical Summary of Numerical Features

**What is happening:** Generating summary statistics (mean, std, min, max) for numerical variables.

**The "Why":** This helps identify the range of values for each feature, which is vital for deciding the scaling strategy.

**Best Practices:** Look for anomalies, such as negative values in asset valuations, that might indicate data entry errors.

In [114]:
df.describe()

       no_of_dependents  income_annum   loan_amount    loan_term  cibil_score  \
count       4269.000000  4.269000e+03  4.269000e+03  4269.000000  4269.000000   
mean           2.498712  5.059124e+06  1.513345e+07    10.900445   599.936051   
std            1.695910  2.806840e+06  9.043363e+06     5.709187   172.430401   
min            0.000000  2.000000e+05  3.000000e+05     2.000000   300.000000   
25%            1.000000  2.700000e+06  7.700000e+06     6.000000   453.000000   
50%            3.000000  5.100000e+06  1.450000e+07    10.000000   600.000000   
75%            4.000000  7.500000e+06  2.150000e+07    16.000000   748.000000   
max            5.000000  9.900000e+06  3.950000e+07    20.000000   900.000000   

       residential_assets_value  commercial_assets_value  luxury_assets_value  \
count              4.269000e+03             4.269000e+03         4.269000e+03   
mean               7.472617e+06             4.973155e+06         1.512631e+07   
std                6.503637

## 3. Data Pre-processing

This section involves splitting the dataset and encoding the target variable to prepare for model ingestion.

### Splitting Data into Training and Testing Sets

**What is happening:** Dividing features (X) and target (Y) into train and test subsets.

**The "Why":** A 20% test split allows us to evaluate the model's accuracy on unseen data.

**Best Practices:** We encode the target variable using **OneHotEncoder** with `drop='if_binary'` to create a single binary column and prevent redundancy.

In [115]:
X_df = df.iloc[:, :-1]
Y_df = df.iloc[:, -1]

# Encode the target variable (Binary Loan Status)
Y_encoder = OneHotEncoder(drop='if_binary', sparse_output=False)
Y = Y_encoder.fit_transform(Y_df.to_frame())

# Split into training (80%) and testing (20%) sets
train_X_df, test_X_df, train_Y, test_Y = train_test_split(X_df, Y, test_size=.2)

### Separating Numerical and Categorical Features

**What is happening:** Isolating different data types for specialized processing.

**The "Why":** Machine learning algorithms generally require numerical input, but categorical variables first need translation via encoding.

In [116]:
train_X_numerical = train_X_df.select_dtypes(include=['number'])
train_X_categorical = train_X_df.select_dtypes(exclude=['number'])

test_X_numerical = test_X_df.select_dtypes(include=['number'])
test_X_categorical = test_X_df.select_dtypes(exclude=['number'])

## 4. Feature Engineering

We apply transformations to numerical and categorical features to ensure optimal model performance.

### Feature Scaling and Encoding

**What is happening:** Standardizing numerical features and One-Hot encoding categorical ones.

**The "Why":** Standardization (calculating $z = \frac{x - \mu}{\sigma}$) ensures that high-magnitude features like 'loan_amount' don't overpower smaller ones like 'no_of_dependents'.

**Best Practices:** Scale only on the training set to prevent **Data Leakage** into the test set.

In [117]:
# Scale numerical features using Z-score standardization
scaler = StandardScaler()
scaler_model = scaler.fit(train_X_numerical)

train_X_num = scaler_model.transform(train_X_numerical)
test_X_num = scaler_model.transform(test_X_numerical)

# Encode categorical features
encoder = OneHotEncoder(sparse_output=False, drop='if_binary')
encoder_model = encoder.fit(train_X_categorical)
train_X_categ = encoder.transform(train_X_categorical)
test_X_categ = encoder.transform(test_X_categorical)

# Recombine into single arrays
train_X = np.concat([train_X_num, train_X_categ], axis=1)
test_X = np.concat([test_X_num, test_X_categ], axis=1)

## 5. Model Training

### Training the Logistic Regression Model

**What is happening:** Fitting the model to the training data.

**The "Why":** **Logistic Regression** is a powerful baseline for classification, offering both reliability and interpretability for banking decisions.

In [118]:
model = LogisticRegression()

# Perform model fitting
# ravel() is used to convert target shape if warnings appear
model.fit(train_X, train_Y.ravel())

## 6. Evaluation & Results

In the final phase, we use several metrics to measure how well our model predicts loan approvals.

### Evaluating Model Performance

**What is happening:** Generating a detailed classification report.

**The "Why":** Accuracy alone is insufficient. We need to monitor Precision and Recall to understand the trade-off between missing a good loan and approving a bad one.

In [119]:
pred_Y = model.predict(test_X)
report = classification_report(test_Y, pred_Y)
print(report)

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       546
         1.0       0.89      0.89      0.89       308

    accuracy                           0.92       854
   macro avg       0.91      0.91      0.91       854
weighted avg       0.92      0.92      0.92       854



### Visualizing the Confusion Matrix

**What is happening:** Plotting a heatmap of the classification errors.

**The "Why":** It clarifies the distribution of True Positives, True Negatives, False Positives (Type I Error), and False Negatives (Type II Error).

In [121]:
con_mat = confusion_matrix(test_Y, pred_Y)
plt.figure(figsize=(10, 10))
heat_map = sns.heatmap(con_mat, annot=True, cmap='viridis', fmt='d')
plt.title('Prediction Heatmap')
plt.show()

## 7. Business Context & Results Interpretation

For financial stakeholders, the model's metrics translate directly into operational efficiency and risk management:

### Precision and Credibility
With a high **Precision** (e.g., ~0.94 for approved loans), the bank minimizes the risk of granting loans to high-risk candidates. This protects the bank's assets from default.

### Recall and Opportunity
A solid **Recall** implies that the bank is capturing a large percentage of creditworthy individuals. High recall means fewer legitimate customers are being rejected, ensuring the bank remains competitive and growth-oriented.

### Business Recommendation
The model shows strong performance with an accuracy of over 90%. However, continuous monitoring of **False Negatives** is recommended, as missing out on low-risk customers represents a loss in potential interest revenue. Future iterations could explore non-linear models like **Random Forests** to potentially improve the F1-score further.